In [212]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from prophet import Prophet
import plotly.express as px
import plotly.graph_objects as go
import warnings 
warnings.filterwarnings('ignore')

## Linear Regression Anomaly Detection Using Z-Scores

In [213]:
#loading data
df_zscore = pd.read_csv('zscore.csv')
df_zscore.head()

,Unnamed: 0,Count,Cumulative DB Time,Cumulative CPU Time,Release,Date
0,0,94714768,2.025239e+10,3.345248e+10,previous,2023-06-08
1,1,10966897,2.211468e+09,3.641333e+09,previous,2023-06-09
2,2,33861629,6.709294e+09,1.113709e+10,previous,2023-06-10
3,3,36862795,6.587202e+09,1.168278e+10,previous,2023-06-11
4,4,98613372,1.916398e+10,3.423117e+10,previous,2023-06-12


In [325]:
#taking a look at the raw data
trace1 = go.Scatter(x = df_zscore['Count'], y = df_zscore['Cumulative DB Time'], mode = 'markers', name = 'Cumulative DB Time')
trace2 = go.Scatter(x=df_zscore['Count'],y=df_zscore['Cumulative CPU Time'],mode='markers', name = 'Cumulative App CPU Time')
go.Figure(data=[trace1,trace2]).update_layout(title = 'Raw Data of Transactions processed vs. Cumulative App CPU Time / Cumulative DB Time',
    xaxis_title = 'Number of Transactions Processed', yaxis_title = 'ms')

In [214]:
#separating data into 2 dataframes based on previous and current releases
df_zscore_prev_release = df_zscore[df_zscore['Release']=='previous']
df_zscore_curr_release = df_zscore[df_zscore['Release']=='current']

In [215]:
#function to calculate R2 value of 2 columns
def calculate_R2(df_zscore_prev_release, metric):
    x=np.array(df_zscore_prev_release['Count'])
    y=np.array(df_zscore_prev_release[metric])
    return np.corrcoef(x,y)[0][1]

In [216]:
calculate_R2(df_zscore_prev_release, 'Cumulative CPU Time')

0.9975049651083954

In [217]:
calculate_R2(df_zscore_prev_release, 'Cumulative DB Time')

0.9930785261012457

### Both metrics have a high correlation to Count (transactions processed) based on the R2 value so we can continue on with our linear regression anomaly detection model.

In [218]:
#calculating zscore values from previous and current releases given a metric of interest
def calculate_zscores(df_zscore_prev_release, df_zscore_curr_release, metric):
    lm = LinearRegression()
    lm.fit(np.array(df_zscore_prev_release['Count']).reshape(-1,1),np.array(df_zscore_prev_release[metric])) 
    predicted_current_release_metric = lm.predict(np.array(df_zscore_curr_release['Count']).reshape(-1,1))
    predicted_previous_release_metric = lm.predict(np.array(df_zscore_prev_release['Count']).reshape(-1,1))
    zscores = (df_zscore_curr_release[metric]-predicted_current_release_metric)/np.std(df_zscore_prev_release[metric]-predicted_previous_release_metric)
    return predicted_current_release_metric, predicted_previous_release_metric, zscores

In [219]:
#testing out calculate_zscore function
metric='Cumulative DB Time'
predicted_current_release_metric, predicted_previous_release_metric, zscores = calculate_zscores(df_zscore_prev_release, df_zscore_curr_release, metric)

In [220]:
#given a zscore threshold of 1, coloring zscores < 1 as green and zscores >= 1 as red
df_zscore_curr_release['zscore']=zscores
def zscore_color(z):
        if z<1:
            return 'green'
        else:
            return 'red'
df_zscore_curr_release['zscore_color']=df_zscore_curr_release['zscore'].apply(zscore_color)


In [303]:
#plotting the linear regression line of the previous release's number of transactions processed and metric of interest
linear_regression_line = go.Scatter(x=df_zscore_prev_release['Count'],y=predicted_previous_release_metric,
    mode='lines',line=dict(color='black',width=1),name='Previous number of transactions and {metric} scatterplot'.format(metric=metric))
go.Figure(data=[linear_regression_line]).update_layout(
    title="Linear regression line of the previous release's number of transactions processed and metric of interest",
    xaxis_title='Number of Transactions Processed', yaxis_title= metric)

In [307]:
#previous days of the current release plotted with number of transactions processed on the x axis and metric of interest on the y axis
df_zscore_today = df_zscore_curr_release.tail(1)
df_zscore_curr_release_days = df_zscore_curr_release.head(-1)
previous_days_of_curr_release=go.Scatter(x=df_zscore_curr_release_days['Count'],
        y=df_zscore_curr_release_days[metric],
        mode='markers',marker_size=8, 
        name= 'Previous release days of current release', 
        hovertext= df_zscore_curr_release_days['Date'],
        marker_color=df_zscore_curr_release_days['zscore_color'])
go.Figure(data=[previous_days_of_curr_release]).update_layout(
    title="Previous days of the current release",
    xaxis_title='Number of Transactions Processed', yaxis_title= metric)


In [308]:
#larger dot for current day of the current release plotted 
today=go.Scatter(x=df_zscore_today['Count'], y=df_zscore_today[metric], 
                  mode='text+markers', marker_size=20, 
                  text=str(np.round(df_zscore_today['zscore'].iloc[0],3)), textposition = 'top center', 
                  hovertext= df_zscore_today['Date'],name = 'Today', 
                  marker_color=df_zscore_today['zscore_color'],
                  marker=dict(size=10, line=dict(width=2, color='black')))
go.Figure(data=[linear_regression_line, previous_days_of_curr_release, today]).update_layout(
    title="Current day of the current release",
    xaxis_title='Number of Transactions Processed', yaxis_title= metric)


In [309]:
def zscore_color(z):
      if z<1:
            return 'green'
      else:
            return 'red'
def graph_zscore(df_zscore_prev_release, df_zscore_curr_release, metric):
      predicted_current_release_metric, predicted_previous_release_metric, zscores = calculate_zscores(df_zscore_prev_release, df_zscore_curr_release, metric)
      df_zscore_curr_release['zscore']=zscores
      df_zscore_curr_release['zscore_color']=df_zscore_curr_release['zscore'].apply(zscore_color)
      linear_regression_line = go.Scatter(x=df_zscore_prev_release['Count'],y=predicted_previous_release_metric,mode='lines',line=dict(color='black',width=1),name='Previous release number of transactions and {metric} scatterplot'.format(metric=metric))
      df_zscore_today = df_zscore_curr_release.tail(1)
      df_zscore_curr_release_days = df_zscore_curr_release.head(-1)
      previous_days_of_curr_release=go.Scatter(x=df_zscore_curr_release_days['Count'],
            y=df_zscore_curr_release_days[metric],
            mode='markers',marker_size=8, 
            name= 'Previous days of current release', 
            hovertext= df_zscore_curr_release_days['Date'],
            marker_color=df_zscore_curr_release_days['zscore_color'])
      today=go.Scatter(x=df_zscore_today['Count'], y=df_zscore_today[metric], 
            mode='text+markers', marker_size=20, 
            text=str(np.round(df_zscore_today['zscore'].iloc[0],3)), textposition = 'top center', 
            hovertext= df_zscore_today['Date'],name = 'Today', 
            marker_color=df_zscore_today['zscore_color'],
            marker=dict(size=10, line=dict(width=2, color='black')))
      graph = go.Figure(data=[linear_regression_line, previous_days_of_curr_release, today])
      graph.update_layout(
            height=500,
            title="Anomaly Detection Comparing Previous and Current Releases: {metric}".format(metric=metric),
            xaxis_title="Number of Transactions Processed",
            yaxis_title="{metric} (ms)".format(metric=metric))
      return graph


In [310]:
graph_zscore(df_zscore_prev_release, df_zscore_curr_release, 'Cumulative CPU Time')

In [311]:
graph_zscore(df_zscore_prev_release, df_zscore_curr_release, 'Cumulative DB Time')

## Anomaly Detection With Prophet

In [312]:
#taking a look at prophet.csv
df_prophet = pd.read_csv('prophet.csv')
df_prophet.drop('Unnamed: 0', axis=1).head()

,Hour,P95 Runtime
0,2023-06-09 00:00:00,1848.46
1,2023-06-09 01:00:00,1828.53
2,2023-06-09 02:00:00,2046.14
3,2023-06-09 03:00:00,1877.21
4,2023-06-09 04:00:00,1642.56


In [328]:
#taking a look at the raw data
trace1 = go.Scatter(x = df_prophet['Hour'], y = df_prophet['P95 Runtime'], mode = 'markers', name = 'P95 Runtime')
go.Figure(data=[trace1]).update_layout(title = 'Raw Data of Hourly P95 Runtime',
    xaxis_title = 'Day / Hour', yaxis_title = 'P95 Runtime (ms)')

In [228]:
#renaming Hour to ds and P95 Runtime to y - the Prophet model identifies these two columns
data = df_prophet.rename(columns={'Hour':'ds','P95 Runtime':'y'}).drop('Unnamed: 0',axis=1)
#changing ds to a datetime column type
data['ds'] = data['ds'].astype('datetime64[ns]')

In [329]:
#fitting the model to the data and using the predict data to produce yhat and confidence interval
model = Prophet(interval_width=0.99, yearly_seasonality=True, weekly_seasonality=True)
model.fit(data)
forecast = model.predict(data)
forecast.head()

14:37:28 - cmdstanpy - INFO - Chain [1] start processing
14:37:29 - cmdstanpy - INFO - Chain [1] done processing


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-06-09 00:00:00,1955.355051,1542.130122,2514.776959,1955.355051,1955.355051,65.069249,65.069249,65.069249,-189.436424,...,81.815857,81.815857,81.815857,172.689816,172.689816,172.689816,0.0,0.0,0.0,2020.424300
1,2023-06-09 01:00:00,1955.287597,1561.317720,2519.711691,1955.287597,1955.287597,85.028784,85.028784,85.028784,-166.302002,...,80.321957,80.321957,80.321957,171.008829,171.008829,171.008829,0.0,0.0,0.0,2040.316380
2,2023-06-09 02:00:00,1955.220142,1554.029500,2568.945780,1955.220142,1955.220142,117.059638,117.059638,117.059638,-130.946218,...,78.666800,78.666800,78.666800,169.339056,169.339056,169.339056,0.0,0.0,0.0,2072.279780
3,2023-06-09 03:00:00,1955.152688,1661.775906,2644.290239,1955.152688,1955.152688,172.827236,172.827236,172.827236,-71.695300,...,76.842044,76.842044,76.842044,167.680492,167.680492,167.680492,0.0,0.0,0.0,2127.979923
4,2023-06-09 04:00:00,1955.085233,1688.508282,2666.704620,1955.085233,1955.085233,239.186268,239.186268,239.186268,-1.685065,...,74.838200,74.838200,74.838200,166.033132,166.033132,166.033132,0.0,0.0,0.0,2194.271501


In [332]:
#using yhat and confidence interval to categorize data as anomalous or normal 
performance = pd.merge(data, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on='ds')
display(performance.head())
performance['anomaly'] = performance.apply(lambda rows: 1 if ((rows.y<rows.yhat_lower)|(rows.y>rows.yhat_upper)) else 0, axis = 1)
anomalies = performance[performance['anomaly']==1].sort_values(by='ds')
display(performance.head())
performance['color'] = np.where(performance['anomaly']== 1, 'red', 'green')
performance['name'] = np.where(performance['anomaly'] == 1, 'Anomaly','Within Confidence Interval')
display(performance.head())

,ds,y,yhat,yhat_lower,yhat_upper
0,2023-06-09 00:00:00,1848.46,2020.424300,1542.130122,2514.776959
1,2023-06-09 01:00:00,1828.53,2040.316380,1561.317720,2519.711691
2,2023-06-09 02:00:00,2046.14,2072.279780,1554.029500,2568.945780
3,2023-06-09 03:00:00,1877.21,2127.979923,1661.775906,2644.290239
4,2023-06-09 04:00:00,1642.56,2194.271501,1688.508282,2666.704620


,ds,y,yhat,yhat_lower,yhat_upper,anomaly
0,2023-06-09 00:00:00,1848.46,2020.424300,1542.130122,2514.776959,0
1,2023-06-09 01:00:00,1828.53,2040.316380,1561.317720,2519.711691,0
2,2023-06-09 02:00:00,2046.14,2072.279780,1554.029500,2568.945780,0
3,2023-06-09 03:00:00,1877.21,2127.979923,1661.775906,2644.290239,0
4,2023-06-09 04:00:00,1642.56,2194.271501,1688.508282,2666.704620,1


,ds,y,yhat,yhat_lower,yhat_upper,anomaly,color,name
0,2023-06-09 00:00:00,1848.46,2020.424300,1542.130122,2514.776959,0,green,Within Confidence Interval
1,2023-06-09 01:00:00,1828.53,2040.316380,1561.317720,2519.711691,0,green,Within Confidence Interval
2,2023-06-09 02:00:00,2046.14,2072.279780,1554.029500,2568.945780,0,green,Within Confidence Interval
3,2023-06-09 03:00:00,1877.21,2127.979923,1661.775906,2644.290239,0,green,Within Confidence Interval
4,2023-06-09 04:00:00,1642.56,2194.271501,1688.508282,2666.704620,1,red,Anomaly


In [333]:
#plotting this hourly data, color-coded to whether it was flagged normal or anomalous
fig = go.Figure()
fig.update_layout(
    height=800,
    title="Anomaly Detection Release over Release",
    xaxis_title="Time",
    yaxis_title="P95 Run Time (ms)",
    
)

fig.add_trace(go.Scatter(x=performance['ds'], y=performance['y'],
                    mode='markers',
                    name='Anomalies - Red Colored Dots',    
                    marker_color=performance.color,
                    
                        ))

In [334]:
#adding the black line, the yhat, the predicted value of the hourly data 
fig = go.Figure()
fig.update_layout(
    height=800,
    title="Anomaly Detection Release over Release",
    xaxis_title="Time",
    yaxis_title="P95 Run Time (ms)",
    
)

fig.add_trace(go.Scatter(x=performance['ds'], y=performance['y'],
                    mode='markers',
                    name='Anomalies - Red Colored Dots',    
                    marker_color=performance.color,
                    
                        ))
fig.add_trace(go.Scatter(x=performance['ds'], y=performance['yhat'],
                    mode='lines',
                    name='p95_runTime',
                    marker_color="black"   
                        ))
fig.show()

## Analysis of Top Customers

In [293]:
#taking a look at customers.csv
df_customers = pd.read_csv('customers.csv')
df_customers = df_customers.drop('Unnamed: 0',axis=1)
df_customers

,Customer,Count Previous,DB Time Previous,CPU Time Previous,Count Current,DB Time Current,CPU Time Current
0,Customer A,9481661,5.303934e+08,8.192375e+08,7585328.80,4.243148e+08,6.553900e+08
1,Customer B,3160958,6.618063e+08,8.474183e+08,2686814.30,5.625353e+08,7.203055e+08
2,Customer C,3149174,1.075398e+09,1.801124e+09,2834256.60,9.678583e+08,1.621011e+09
3,Customer D,2929447,6.327836e+08,9.509118e+08,2782974.65,6.011444e+08,9.033662e+08
4,Customer E,2519622,3.209520e+08,4.613936e+08,2519622.00,3.209520e+08,4.613936e+08
5,Customer F,2446161,5.123145e+07,3.096698e+08,2568469.05,5.379302e+07,3.251533e+08
6,Customer G,2291756,5.134051e+08,6.661722e+08,2520931.60,5.647456e+08,7.327895e+08
7,Customer H,2273342,4.603147e+08,9.738171e+08,2614343.30,5.293619e+08,1.119890e+09
8,Customer I,2096236,2.869418e+08,4.325658e+08,2515483.20,3.443301e+08,5.190789e+08
9,Customer J,1996811,9.928914e+07,1.067002e+08,2496013.75,1.241114e+08,1.333752e+08


In [340]:
trace1 = go.Bar(x = df_customers['Customer'], y=df_customers['Count Previous'], name = 'Count Previous')
trace2 = go.Bar(x = df_customers['Customer'], y=df_customers['Count Current'], name = 'Count Current')
trace3 = go.Bar(x = df_customers['Customer'], y=df_customers['DB Time Previous'], name = 'DB Time Previous')
trace4 = go.Bar(x = df_customers['Customer'], y=df_customers['DB Time Current'], name = 'DB Time Current')
trace5 = go.Bar(x = df_customers['Customer'], y=df_customers['CPU Time Previous'], name = 'CPU Time Previous')
trace6 = go.Bar(x = df_customers['Customer'], y=df_customers['CPU Time Current'], name = 'CPU Time Current')
go.Figure(data=[trace1, trace2, trace3, trace4, trace5, trace6]).update_layout(
    title = 'Customer Data'
)


In [294]:
#a function to calculate percentage change
def percentage_change(previous,current):
    return np.round(((current-previous)*100/previous),2)

In [295]:
#applying the percentage change function to DB and app CPU time for each customer
df_customers['DB Percent Change']=percentage_change(df_customers['DB Time Previous'],df_customers['DB Time Current'])
df_customers['CPU Percent Change']=percentage_change(df_customers['CPU Time Previous'],df_customers['CPU Time Current'])

In [296]:
#a function that categorizes customers in different colors based on the percent change of their DB and app CPU times 
def customer_color(db, cpu):
    color=[]
    for n in range(len(db)):
        if db[n] < 0 and cpu[n] < 0:
            color.append('green')
        elif db[n] < 5 and cpu[n] < 5:
            color.append('yellow')
        else:
            color.append('red')
    return color

In [313]:
#applying the function that categorizes customers into colors
df_customers['Customer Color'] = customer_color(df_customers['DB Percent Change'], df_customers['CPU Percent Change'])

In [298]:
#graphing the percent change of these resource utilization metrics in the past week for top customers 
fig = px.scatter(df_customers, color='Customer Color', x='DB Percent Change', 
    y='CPU Percent Change', size='Count Current', size_max = 50, 
    color_discrete_map={"red": "red", "yellow": "yellow", "green": "green"}, 
    text='Customer')
fig.update_layout(title = 'Percent Change in sum(dbTime) and sum(cpuTime) in the past week',
    xaxis_title = 'Percent Change in sum(dbTime)', yaxis_title = 'Percent Change in sum(cpuTime)')
fig.show()